In [1]:
import numpy as np
from collections import deque 

In [2]:
def get_lines(fname):
    values: dict = get_values(fname)
    positions: list = get_positions(fname)
    with open("control_section","r") as f:
        l = f.readlines() 
    lines = [_.format(**values) for _ in l]
    with open("system_section","r") as f:
        l = f.readlines()
    lines+=[_.format(**values) for _ in l]
    with open("electrons_section","r") as f:
        l = f.readlines()
    lines+=[_.format(**values) for _ in l]
    with open("cards","r") as f: 
        l=f.readlines() 
    i0=0 
    s = deque(filter(lambda e: "ATOMIC_POSITIONS" in e,l),maxlen=1)[0]
    i1 = l.index(s)
    s = deque(filter(lambda e: "K_POINTS" in e, l),maxlen=1)[0]
    i2 = l.index(s)
    lines += l[i0:i1+1]
    ll = lambda pos: (f"{_[0]}   {_[1][0]:12.5f}   {_[1][1]:12.5f} {_[1][2]:12.5f} \n" for _ in pos)
    lines+=list(deque(ll(positions)))
    lines+=l[i2:i2+1]
    return lines 

def get_values(fname):
    calculation_id: str = fname.replace('.angst','')
    with open(fname,'r') as f:
        alat: float = float(next(f).split()[2])
        nat:  int   = int (next(f).split()[2])
    return {'calculation_id':calculation_id,'alat_angst':alat, 'nat':nat}


def xyzline(s):
    f=s.split()
    if len(f) < 4:
        return None
    symb=f[0]
    coords=np.array([float(x) for x in f[1:4]])
    return (symb, coords)


def get_positions(fname):
    with open(fname,"r") as f:
        res= [_ for _ in filter(None,(xyzline(__)for __ in f))]
    return res

def write_input(fname):
    inpfile: str = fname.replace('angst','in')
    with open(inpfile,'w') as fin:
        deque ((fin.write(_) for _ in get_lines(fname)))


In [13]:
write_input('011.angst')

